## Exemplos para implementação de Data Pipelines

### Download do arquivo de síndromes gripais do OpenDataSUS

In [3]:
!wget https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SGL/2020/uf=PB/lote=1/part-00000-0508a0cd-d93a-42d4-b285-50673d466ef5.c000.csv



--2022-11-23 19:26:19--  https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SGL/2020/uf=PB/lote=1/part-00000-0508a0cd-d93a-42d4-b285-50673d466ef5.c000.csv
Resolving s3.sa-east-1.amazonaws.com (s3.sa-east-1.amazonaws.com)... 52.95.165.88
Connecting to s3.sa-east-1.amazonaws.com (s3.sa-east-1.amazonaws.com)|52.95.165.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261065782 (249M) [text/csv]
Saving to: 'part-00000-0508a0cd-d93a-42d4-b285-50673d466ef5.c000.csv'

part-00000-0508a0cd 100%[===================>] 248.97M  36.4MB/s    in 8.0s    

2022-11-23 19:26:27 (31.2 MB/s) - 'part-00000-0508a0cd-d93a-42d4-b285-50673d466ef5.c000.csv' saved [261065782/261065782]



In [4]:
!ls

Projeto Integração de Dados.ipynb
Untitled.ipynb
part-00000-0508a0cd-d93a-42d4-b285-50673d466ef5.c000.csv


In [5]:
!mv part-00000-0508a0cd-d93a-42d4-b285-50673d466ef5.c000.csv sindrome-gripal.csv

In [6]:
!ls

Projeto Integração de Dados.ipynb   sindrome-gripal.csv
Untitled.ipynb


In [7]:
!pip install pandas

In [29]:
import pandas as pd

df = pd.read_csv('sindrome-gripal.csv', sep=";", encoding='utf-8', low_memory=False)
df.fillna('', inplace=True)
df.head()

,source_id,dataNotificacao,dataInicioSintomas,sintomas,profissionalSaude,racaCor,outrosSintomas,outrasCondicoes,profissionalSeguranca,cbo,...,codigoTipoTeste3,codigoFabricanteTeste3,codigoResultadoTeste3,dataColetaTeste3,codigoEstadoTeste4,codigoTipoTeste4,codigoFabricanteTeste4,codigoResultadoTeste4,dataColetaTeste4,idade
0,aA4Owws0YG,2020-06-10,2020-06-02,"Dor de Cabeça, Tosse, Febre, Dor de Garganta",Não,Parda,,,Não,,...,,,,,,,,,,59.0
1,LDtfPeHDuC,2020-07-14,2020-07-07,"Febre, Tosse",Não,Branca,,,Não,,...,,,,,,,,,,34.0
2,rPKXWXx07G,2020-11-27,2020-11-20,"Coriza, Dor de Cabeça, Tosse, Febre, Dor de Ga...",Não,Branca,,,Não,,...,,,,,,,,,,27.0
3,XDH3Q3tLh4,2020-07-22,2020-07-10,"Dor de Garganta, Tosse",Não,Branca,,,Sim,,...,,,,,,,,,,39.0
4,l1UvBys4aK,2020-12-01,2020-11-28,"Febre, Dispneia, Dor de Garganta",Não,Parda,,,Não,,...,,,,,,,,,,24.0


In [32]:
!pip install minio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 482.8 kB/s eta 0:00:001m685.9 kB/s eta 0:00:01


## Preparando conexão com o MinIO

In [67]:
from minio import Minio

client = Minio("localhost:9000", secure=False, access_key="FFD34B2AC56E76E8BB9E7EFD7D283",
    secret_key="47A13EFAB1CA1F253FD6F56DEF769")

existing_buckets = client.list_buckets()
existing_buckets = [i.name for i in existing_buckets]
if "pendentes" not in existing_buckets:
    client.make_bucket("pendentes")
if "processados" not in existing_buckets:
    client.make_bucket("processados")
client.list_buckets()

[Bucket('baixados'), Bucket('pendentes'), Bucket('processados')]

In [38]:
!pwd
!ls

/Users/diegopessoa/projects/ifpb/integracao-dados/notebooks
Projeto Integração de Dados.ipynb   sindrome-gripal.csv
Untitled.ipynb                      sindrome-gripal.xls


## Enviando arquivo para o MinIO

In [39]:
client.fput_object("pendentes", "sindrome-gripal.csv", "/Users/diegopessoa/projects/ifpb/integracao-dados/notebooks/sindrome-gripal.csv")

## Baixando arquivo do MinIO

In [68]:
from io import BytesIO

arquivo = client.get_object("baixados", "movies.csv")
df = pd.read_csv(BytesIO(arquivo.read()))
df.head()

                                 Film    Genre            Lead Studio  \
0          Zack and Miri Make a Porno  Romance  The Weinstein Company   
1                     Youth in Revolt   Comedy  The Weinstein Company   
2  You Will Meet a Tall Dark Stranger   Comedy            Independent   
3                        When in Rome   Comedy                 Disney   
4               What Happens in Vegas   Comedy                    Fox   

   Audience score %  Profitability  Rotten Tomatoes % Worldwide Gross  Year  
0                70       1.747542                 64         $41.94   2008  
1                52       1.090000                 68         $19.62   2010  
2                35       1.211818                 43         $26.66   2010  
3                44       0.000000                 15         $43.04   2010  
4                72       6.267647                 28        $219.37   2008  
